In [1]:
from pathlib import Path
import pandas as pd
import numpy as np


from reportengine.compat import yaml
import validphys

# Path to your buildmaster, here I'm guessing vp as installed in edit mode
BUILDMASTER_PATH = Path(validphys.__file__).parent / "../../../buildmaster"
DATASET = "NMCPD"

#from validphys.core import CommonDataSpec
#import pandas as pd
#import numpy as np
#from collections import defaultdict

dataset_path = BUILDMASTER_PATH / DATASET
metadata_file = dataset_path / "metadata.yaml"
metadata = yaml.safe_load(metadata_file.read_text())
ndata = metadata["ndata"]

In [2]:
# Read the kinematics dataframe
kin_df = None
if True:
    kin_file = dataset_path / "kinematics.yaml"
    kinyaml = yaml.safe_load(kin_file.read_text())
    
    kin_dict = {i+1: pd.DataFrame(d).stack() for i, d in enumerate(kinyaml["bins"])}
    kin_df = pd.concat(kin_dict, axis=1, names=["index"]).swaplevel(0,1).T
kin_df

,x,q2,y
,mid,mid,mid
index,,,
1,0.0015,0.16,0.120852
2,0.0015,0.25,0.188831
3,0.0015,0.35,0.264363
4,0.0015,0.45,0.339895
5,0.0015,0.60,0.453194
...,...,...,...
256,0.6750,26.49,0.044463
257,0.6750,35.40,0.059419


In [5]:
# Read the kinematics dataframe
variant = True
unc_df = None

def read_uncertainties(unc_file):
    uncyaml = yaml.safe_load(unc_file.read_text())
    
    mindex = pd.MultiIndex.from_tuples([(k, v["treatment"], v["type"]) for k,v in uncyaml["definition"].items()], names=["name", "treatment", "type"])
    # I'm guessing there will be a better way of doing this than calling  dataframe twice for the same thing
    final_df = pd.DataFrame(pd.DataFrame(uncyaml["bins"]).values, columns=mindex, index=range(1,ndata+1))
    final_df.index.name = "index"
    return final_df 

if True:
    unc_df = read_uncertainties(dataset_path / "uncertainties.yaml")
    
    if variant:
        # With a variant
        var_df = read_uncertainties(dataset_path / "uncertainties_dw.yaml")
        
        unc_df = pd.concat([unc_df, var_df], axis=1)
        
unc_df

name,stat_1,syst_1,nuclear
treatment,ADD,ADD,ADD
type,UNCORR,CORR,NUC_DW
index,,,
1,0.0203,0.0109,0.0203
2,0.0212,0.0134,0.0212
3,0.0205,0.0112,0.0205
4,0.0258,0.0195,0.0258
5,0.0176,0.0121,0.0176
...,...,...,...
256,0.0235,0.0034,0.0235


In [4]:
data_df = None
    
if True:
    data_file = dataset_path / "data.yaml"
    datayaml = yaml.safe_load(data_file.read_text(encoding="utf-8"))
    
    data_df = pd.DataFrame(datayaml["data_central"], index=range(1, ndata+1), columns=["data"])
    data_df.index.name = "index"
    
data_df

,data
index,
1,0.9815
2,1.0030
3,0.9675
4,1.0330
5,0.9912
...,...
256,0.6717
257,0.7194
258,0.6959
